<a href="https://colab.research.google.com/github/2303A51086/AIML-2025/blob/main/Lab03_AIML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Install the python Libraries for Game Strategy
2. Implement a Game Class Constructor with-actions,is_terminal,result
3. Implement a TicTocToe Player Game using game class Function

In [1]:
from collections import namedtuple, Counter, defaultdict
import random
import math
import functools
cache=functools.lru_cache(10**6)

In [8]:
 class Game:
  """A game is similar"""



  def actions(self, state):
      """Return a collection of the allowable moves from this state."""
      raise NotImplementedError
 def result(self, state, move):
    """Return the state that results from making a move from a state."""
    raise NotImplementedError
 def is_terminal(self, state):
    """Return True if this is a final state for the game."""
    return not self.actions(state)
 def utility(self, state, player):
  """Return the value of this final state to player."""
  raise NotImplementedError

In [ ]:
def play game(game, strategies: dict, verbose=False):
 Play a turn−taking game. strategies is a player name: function dict,
 where function(state, game) is used to get the player s move.
 state = game.initial
 while not game.is terminal(state):
 player = state.to move
 move = strategies[player](game, state)
 state = game.result(state, move)
 if verbose:
 print( Player , player, move: , move)
 print(state)
 return state

Part-2: Implement the Game Strategy Algorithms.

In [23]:
def minimax_search(game, state):
    """Search game tree to determine best move; return (value, move) pair."""

    player = state.to_move
    infinity = math.inf

    def max_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a))
            if v2 > v:
                v, move = v2, a
        return v, move

    def min_value(state):
	#H
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a))
            if v2 < v:
                v, move = v2, a#2R
        return v, move

    return max_value(state)


In [24]:
def alphabeta_search(game, state):
    """Search game to determine best action; use alpha-beta pruning."""
    player = state.to_move()

    def max_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -float('inf'), None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta)
            if v2 > v:
                v, move = v2, a
            alpha = max(alpha, v)
            if v >= beta:
                return v, move
        return v, move

    def min_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = float('inf'), None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta)
            if v2 < v:
                v, move = v2, a
            beta = min(beta, v)
            if v <= alpha:
                return v, move
        return v, move

    return max_value(state, -float('inf'), float('inf'))[1]

Part 3- Implement the Game Strategy using TicTocToe

In [25]:
class TicTacToe:
    """Play TicTacToe on a 'height' by 'width' board, needing 'k' in a row to win.
    'X' plays first against 'O'."""

    def _init_(self, height=3, width=3, k=3):
        self.k = k  # k in a row
        self.squares = {(x, y) for x in range(width) for y in range(height)}
        self.initial = Board(height=height, width=width, to_move='X', utility=0)

    def actions(self, board):
        """Legal moves are any square not yet taken."""
        return self.squares - set(board.state.keys())

    def result(self, board, square):
        """Place a marker for the current player on the square."""
        player = board.to_move
        new_board_state = board.state.copy()
        new_board_state[square] = player
        new_board = Board(height=board.height, width=board.width, state=new_board_state,
                          to_move=('O' if player == 'X' else 'X'), utility=0)
        win = self.k_in_row(new_board, player, square, self.k)
        new_board.utility = (0 if not win else +1 if player == 'X' else -1)
        return new_board

    def utility(self, board, player):
        """Return the value to the player; 1 for win, -1 for loss, 0 otherwise."""
        return board.utility if player == 'X' else -board.utility

    def is_terminal(self, board):
        """A board is a terminal state if it is won or there are no empty squares."""
        return board.utility != 0 or len(self.squares) == len(board.state)

    def display(self, board):
        """Print the board."""
        for y in range(board.height):
            row = ''
            for x in range(board.width):
                row += board.state.get((x, y), '.') + ' '
            print(row)
        print()

    def k_in_row(self, board, player, square, k):
        """True if player has k pieces in a line through square."""

        def in_row(x, y, dx, dy):
            return 0 if board.state.get((x, y)) != player else 1 + in_row(x + dx, y + dy, dx, dy)

        return any(
            in_row(*square, dx, dy) + in_row(*square, -dx, -dy) - 1 >= k
            for (dx, dy) in ((0, 1), (1, 0), (1, 1), (1, -1))
        )

class Board:
    def _init_(self, height, width, state=None, to_move='X', utility=0):
        self.height = height
        self.width = width
        self.state = state or {}
        self.to_move = to_move
        self.utility = utility

    def _str_(self):
        result = []
        for y in range(self.height):
            row = ''
            for x in range(self.width):
                row += self.state.get((x, y), '.') + ' '
            result.append(row.strip())
        return '\n'.join(result)

In [26]:
from collections import defaultdict

class Board(defaultdict):
    """A board has the player to move, a cached utility value,
    and a dict of {(x, y): player} entries, where player is 'X' or 'O'."""

    empty = '.'
    off = '#'

    def _init_(self, width=8, height=8, to_move=None, **kwds):
        super()._init_(self.missing)  # Initialize defaultdict with the custom missing method
        self.width = width
        self.height = height
        self.to_move = to_move
        self.update(kwds)

    def new(self, changes: dict, **kwds) -> 'Board':
        """Given a dict of {(x, y): contents} changes, return a new Board with the changes."""
        board = Board(width=self.width, height=self.height, to_move=self.to_move, **kwds)
        board.update(self)
        board.update(changes)
        return board

    def missing(self, loc):
        x, y = loc
        if 0 <= x < self.width and 0 <= y < self.height:
            return self.empty
        else:
            return self.off

    def _hash_(self):
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)

    def _repr_(self):
        def row(y): return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(map(row, range(self.height))) + '\n'